# Examples

The following examples show how to use the loader APIs:

## 1. Parsing

This sample shows how to read a CWL document from a remote public URL:

In [ ]:
from cwl_loader import load_cwl_from_location

cwl_document = load_cwl_from_location('https://raw.githubusercontent.com/eoap/ogc-api-processes-with-zoo/refs/heads/feature-EOEPCA-469/cwl-workflows/eoap-api-cli.cwl')

2025-08-11 14:33:58.236 | INFO     | cwl_loader:load_cwl_from_location:186 - Loading CWL document from https://raw.githubusercontent.com/eoap/schemas/main/experimental/stac-api-client.cwl...
2025-08-11 14:33:58.424 | INFO     | cwl_loader:load_cwl_from_yaml:117 - Updating the model to v1.2...
2025-08-11 14:33:58.429 | INFO     | cwl_loader:load_cwl_from_yaml:128 - Raw CWL document successfully updated to v1.2! Now converting to the CWL model...
2025-08-11 14:34:09.071 | INFO     | cwl_loader:load_cwl_from_yaml:136 - Raw CWL document successfully updated to v1.2! Now dereferencing the FQNs...
2025-08-11 14:34:09.072 | INFO     | cwl_loader:_clean_process:62 -   Cleaning CommandLineTool stac-client...
2025-08-11 14:34:09.073 | INFO     | cwl_loader:load_cwl_from_yaml:144 - CWL document successfully dereferenced!


## 2. Serializing

This sample shows how to write a CWL document to a stream (string, file, ...):

In [ ]:
from cwl2puml import (
    to_puml,
    DiagramType
)
from sys import stderr

to_puml(
    cwl_document=cwl_document,
    diagram_type=DiagramType.COMPONENTS,
    output_stream=stderr
)

TypeError: 'CommandLineTool' object is not iterable